<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MCP_CLAUDE4_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_anthropic -q
!pip install crewai -q
!pip install pydantic -q

!pip install anthropic -q
!pip install colab-env -q

In [ ]:
import colab_env

In [3]:
import os
import anthropic
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool
from langchain_anthropic import ChatAnthropic
from pydantic import BaseModel, Field


# Configuration: Retrieve API Key
# Assuming colab_env has already been run in a previous cell
api_key = os.environ.get("CLAUDE3_API_KEY")

if not api_key:
    raise ValueError("CLAUDE3_API_KEY environment variable not set. Please set it (e.g., export CLAUDE3_API_KEY='sk-ant...')")

# Define the LLMs for each agent using langchain_anthropic
opus_llm = ChatAnthropic(model="claude-opus-4-20250514", temperature=0.7)
sonnet_llm = ChatAnthropic(model="claude-sonnet-4-20250514", temperature=0.7)

# Custom Tools (keeping definitions from the original document, adding more specific simulated data for faster results)
class WeatherQueryInput(BaseModel):
    query: str = Field(description="A specific flight identifier or airport ICAO code for which weather is needed.")

class NOTAMQueryInput(BaseModel):
    query: str = Field(description="A specific airport ICAO code or a route for NOTAMs.")

class FlightInfoInput(BaseModel):
    flight_info: str = Field(description="A string containing essential flight details for a quick summary.")

class DetailedWeatherTool(BaseTool):
    name: str = "Detailed Weather Fetcher"
    description: str = "Fetches comprehensive, granular weather data."
    args_schema: BaseModel = WeatherQueryInput
    def _run(self, query: str) -> str:
        # Simulate quick responses for specific queries
        if "KLAX" in query.upper():
            return "Simulated detailed weather for KLAX: Winds 250@12kts, visibility 10sm, few clouds at 8000ft, Temp 18C, Dew 10C. No significant weather."
        elif "KJFK" in query.upper():
            return "Simulated detailed weather for KJFK: Winds 090@8kts, visibility 5sm in light rain, overcast at 1500ft, Temp 12C, Dew 11C. Instrument conditions possible."
        else:
            return f"Simulated detailed weather for {query}: Generally good conditions. Check specifics."

class NOTAMFetcherTool(BaseTool):
    name: str = "NOTAM Fetcher"
    description: str = "Retrieves active Notices to Airmen (NOTAMS)."
    args_schema: BaseModel = NOTAMQueryInput
    def _run(self, query: str) -> str:
        # Simulate quick responses for specific queries
        if "KLAX" in query.upper():
            return "Simulated NOTAM KLAX: Runway 24L/R closed for maintenance until 0500Z. Expect taxi delays."
        elif "KJFK" in query.upper():
            return "Simulated NOTAM KJFK: Tower frequency 121.35 temporarily out of service. Use 121.9. "
        else:
            return f"Simulated NOTAM for {query}: No critical active NOTAMs found."

class QuickWeatherSummaryTool(BaseTool):
    name: str = "Quick Weather Summary"
    description: str = "Provides a very brief, high-level summary of critical weather elements."
    args_schema: BaseModel = FlightInfoInput
    def _run(self, flight_info: str) -> str:
        if "KLAX" in flight_info.upper():
            return "Quick weather KLAX: Clear, mild winds."
        elif "KJFK" in flight_info.upper():
            return "Quick weather KJFK: Light rain, low clouds expected."
        else:
            return f"Quick weather {flight_info}: Overall fair."


# Instantiate tools
detailed_weather_tool = DetailedWeatherTool()
notam_fetcher_tool = NOTAMFetcherTool()
quick_weather_tool = QuickWeatherSummaryTool()


# --- Define Agents ---

# Risk Assessment Agent (Opus-powered)
# Decreased verbosity for faster execution by reducing logging output.
risk_assessment_agent = Agent(
    role="Aviation Safety Analyst",
    goal="Identify and assess potential flight hazards for flight AZ123 from current operational data.",
    backstory="You are an expert in aviation safety, quickly pinpointing risks.",
    llm=opus_llm,
    tools=[notam_fetcher_tool, detailed_weather_tool], # Provide tools to gather data for risk assessment
    verbose=False, # Set to False for faster runs
    allow_delegation=True
)

# Flight Briefing Agent (Sonnet-powered)
# Decreased verbosity for faster execution by reducing logging output.
flight_briefing_agent = Agent(
    role="Pilot Briefing Specialist",
    goal="Provide concise, critical pilot briefings for flight AZ123.",
    backstory="You are a fast and efficient briefing specialist.",
    llm=sonnet_llm,
    tools=[quick_weather_tool],
    verbose=False, # Set to False for faster runs
    allow_delegation=True
)

# Context Synthesizer Agent (Opus-powered)
# This agent's role is simplified for faster execution, focusing on direct data gathering and synthesis.
# Decreased verbosity for faster execution by reducing logging output.
context_synthesizer_agent = Agent(
    role="Flight Data Integrator",
    goal="Quickly gather and synthesize essential weather and NOTAM data for specified airports/routes.",
    backstory="You efficiently pull and combine critical flight data.",
    llm=opus_llm,
    tools=[detailed_weather_tool, notam_fetcher_tool],
    verbose=False, # Set to False for faster runs
    allow_delegation=True
)


# --- Define Tasks ---

# An easy task for the Flight Briefing Agent (from original notebook, kept for comparison)
easy_fun_fact_task = Task(
    description=(
        "Tell me a concise and interesting fun fact about aviation (e.g., 'The first flight was X, lasting Y.')."
    ),
    expected_output="A single, concise, and fascinating aviation fun fact.",
    agent=flight_briefing_agent,
)

# --- Modified Tasks for faster execution ---

# 1. Consolidated Data Collection & Initial Risk Identification Task
# Combining context gathering and initial risk identification to reduce LLM calls and task overhead.
# The agent's description is more direct.
initial_briefing_and_risk_task = Task(
    description=(
        "For flight AZ123 (KLAX to KJFK, dep 2025-06-15), gather detailed weather for KLAX and KJFK, "
        "and all NOTAMs affecting KLAX, KJFK, and the general route. "
        "Based on this data, immediately identify any *critical* weather hazards (e.g., severe storms, low visibility) "
        "or *critical* NOTAM-related issues (e.g., runway closures impacting primary operations). "
        "Output a concise summary structured as: 'Weather: [summary]. NOTAMs: [summary]. Critical Risks: [list].'"
    ),
    expected_output="A concise summary of weather, NOTAMs, and any critical immediate risks for flight AZ123.",
    agent=context_synthesizer_agent, # Using this agent for both data and initial risk scan
    tools=[detailed_weather_tool, notam_fetcher_tool]
)

# 2. Final Briefing Compilation Task
# This task now takes the output of the combined task and formats it into the final briefing.
# Less complex as the primary data gathering and initial risk scan happened in the previous step.
final_briefing_compilation_task = Task(
    description=(
        "From the provided data (weather, NOTAMs, critical risks), compile a professional and concise pilot briefing for flight AZ123. "
        "Prioritize safety-critical information. Format as a pre-flight briefing. "
        "Start with 'PILOT BRIEFING FOR FLIGHT AZ123 (KLAX-KJFK):'."
    ),
    expected_output="A complete, professional, and concise pilot briefing for flight AZ123.",
    agent=flight_briefing_agent, # The briefing specialist is best for final formatting
)


# --- Create the Crew ---

# Crew for the easy fun fact task (unchanged)
fun_fact_crew = Crew(
    agents=[flight_briefing_agent],
    tasks=[easy_fun_fact_task],
    process=Process.sequential,
    verbose=False
)

# MODIFIED: Crew for flight planning with fewer tasks and reduced verbosity
flight_planning_crew_mcp = Crew(
    agents=[context_synthesizer_agent, risk_assessment_agent, flight_briefing_agent], # Keep all agents if their roles are distinct
    tasks=[initial_briefing_and_risk_task, final_briefing_compilation_task], # Reduced to 2 main tasks
    process=Process.sequential,
    verbose=False # Set to False for faster execution
)


import time
import litellm
from litellm.exceptions import InternalServerError # Import the specific exception


# --- Run the Crew ---
if __name__ == "__main__":
    print("\n" + "#" * 80)
    print("## Initiating Fun Fact Generation ##".center(76))
    print("#" * 80 + "\n")

    crew_output_fun_fact = None
    retries = 3 # Number of retries
    delay = 5 # Delay in seconds between retries

    for i in range(retries):
        try:
            crew_output_fun_fact = fun_fact_crew.kickoff()
            break # Exit the loop if successful
        except InternalServerError as e: # Catch the specific Anthropic overloaded error
            print(f"Attempt {i+1} failed: {e}")
            if i < retries - 1:
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
            else:
                print("Max retries reached. Failed to generate fun fact.")
                # Optionally re-raise the exception if you want the script to stop
                # raise e
        except Exception as e: # Catch any other unexpected errors
            print(f"Attempt {i+1} failed with unexpected error: {e}")
            if i < retries - 1:
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
            else:
                print("Max retries reached. Failed to generate fun fact.")
                # Optionally re-raise the exception if you want the script to stop
                # raise e


    if crew_output_fun_fact: # Check if the kickoff was successful
        if hasattr(crew_output_fun_fact, 'tasks_outputs') and crew_output_fun_fact.tasks_outputs:
            fun_fact_result = crew_output_fun_fact.tasks_outputs[-1].result
        elif hasattr(crew_output_fun_fact, 'final_answer'):
            fun_fact_result = crew_output_fun_fact.final_answer
        else:
            fun_fact_result = str(crew_output_fun_fact)

        print("\n" + "#" * 80)
        print("## FINAL FUN FACT RESULT ##".center(76))
        print("#" * 80)
        print(fun_fact_result)
    else:
        print("\n" + "#" * 80)
        print("## FUN FACT GENERATION FAILED ##".center(76))
        print("#" * 80)
        print("Could not generate the fun fact after multiple retries.")


    print("\n" + "=" * 80)
    print("## Initiating Faster Comprehensive Flight Planning (MCP Principles Applied) ##".center(76))
    print("=" * 80 + "\n")

    # Run the flight planning crew
    # Apply the same retry logic to the second kickoff
    crew_output_flight_plan = None
    retries = 3 # Number of retries
    delay = 5 # Delay in seconds between retries

    for i in range(retries):
        try:
            crew_output_flight_plan = flight_planning_crew_mcp.kickoff()
            break # Exit the loop if successful
        except InternalServerError as e: # Catch the specific Anthropic overloaded error
            print(f"Attempt {i+1} failed: {e}")
            if i < retries - 1:
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
            else:
                print("Max retries reached. Failed to generate flight plan.")
                # Optionally re-raise the exception if you want the script to stop
                # raise e
        except Exception as e: # Catch any other unexpected errors
            print(f"Attempt {i+1} failed with unexpected error: {e}")
            if i < retries - 1:
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
            else:
                print("Max retries reached. Failed to generate flight plan.")
                # Optionally re-raise the exception if you want the script to stop
                # raise e


    print("\n" + "=" * 80)
    print("## FINAL FLIGHT BRIEFING RESULT (MCP-Enhanced) ##".center(76))
    print("=" * 80)

    if crew_output_flight_plan: # Check if the kickoff was successful
        if hasattr(crew_output_flight_plan, 'tasks_outputs') and crew_output_flight_plan.tasks_outputs:
            final_briefing_result = crew_output_flight_plan.tasks_outputs[-1].result
        elif hasattr(crew_output_flight_plan, 'final_answer'):
            final_briefing_result = crew_output_flight_plan.final_answer
        else:
            final_briefing_result = str(crew_output_flight_plan)

        print(final_briefing_result)
    else:
        print("\n" + "=" * 80)
        print("## FLIGHT BRIEFING GENERATION FAILED ##".center(76))
        print("=" * 80)
        print("Could not generate the flight briefing after multiple retries.")


################################################################################
                    ## Initiating Fun Fact Generation ##                    
################################################################################


################################################################################
                        ## FINAL FUN FACT RESULT ##                         
################################################################################
The Wright brothers' first powered flight on December 17, 1903, lasted only 12 seconds and covered 120 feet - shorter than the wingspan of a modern Boeing 747.

## Initiating Faster Comprehensive Flight Planning (MCP Principles Applied) ##


             ## FINAL FLIGHT BRIEFING RESULT (MCP-Enhanced) ##              
PILOT BRIEFING FOR FLIGHT AZ123 (KLAX-KJFK):

**DEPARTURE AIRPORT (KLAX):**
- Weather: Winds 250°@12kts, visibility 10sm, few clouds 8,000ft, temp 18°C/64°F, dewpoint 10°C
- Conditions: VFR, no signif